In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
colors = [
    "#FF6347", "#FF4500", "#00FFFF", "#1E90FF", "#4169E1", 
    "#00FA9A", "#40E0D0", "#FF8C00", "#FFA500", "#FFD700",
    "#FF1493", "#D2691E",  "#800000", "#006400", "#228B22",
    "#FF0000", "#8B0000", "#8B4513", "#A52A2A", "#B22222", 
    "#FFFF00", "#9ACD32", "#F08080", "#FF7F50", "#FF6347", 
    "#6B8E23", "#808000", "#DDA0DD", "#EE82EE", "#FF00FF", 
    "#FF69B4", "#C71585", "#32CD32", "#98FB98", "#00FF7F", 
    "#8A2BE2", "#9B30FF", "#FF00FF", "#C71585", "#DC143C", 
    "#F4A460", "#D2B48C", "#BC8F8F", "#F5DEB3", "#FF4500", 
    "#7CFC00", "#00FA9A", "#40E0D0", "#00CED1", "#48D1CC", 
    "#20B2AA", "#008B8B", "#3CB371", "#2E8B57", "#9ACD32", 
    "#6B8E23", "#808000", "#C71585", "#8A2BE2", "#4B0082", 
    "#9400D3", "#9932CC", "#8B008B", "#DDA0DD", "#EE82EE", 
    "#F0E68C", "#BDB76B", "#A52A2A", "#FF0000", "#B22222", 
    "#8B0000", "#800000", "#FFD700", "#FFFF00", "#9ACD32", 
    "#32CD32", "#98FB98", "#00FF7F", "#00FA9A", "#40E0D0", 
    "#00FFFF", "#1E90FF", "#4169E1", "#8A2BE2", "#9B30FF", 
    "#FF00FF", "#C71585", "#DC143C", "#FF1493", "#D2691E", 
    "#8B4513", "#A52A2A", "#B22222", "#FF0000", "#8B0000", 
    "#800000", "#006400", "#228B22", "#6B8E23", "#808000", 
    "#DDA0DD", "#EE82EE", "#FF00FF", "#FF69B4", "#C71585", 
    "#F08080", "#FF7F50", "#FF6347", "#F4A460", "#D2B48C", 
    "#BC8F8F", "#F5DEB3", "#FF4500", "#7CFC00", "#00FA9A", 
    "#40E0D0", "#00CED1", "#48D1CC", "#20B2AA", "#008B8B", 
    "#3CB371", "#2E8B57", "#9ACD32", "#6B8E23", "#808000", 
    "#C71585", "#8A2BE2", "#4B0082", "#9400D3", "#9932CC", 
    "#8B008B", "#DDA0DD", "#EE82EE", "#F0E68C", "#BDB76B", 
    "#A52A2A", "#FF0000", "#B22222", "#8B0000", "#800000", 
    "#FFD700", "#FFFF00", "#9ACD32", "#32CD32", "#98FB98", 
    "#00FF7F", "#00FA9A", "#40E0D0", "#00FFFF", "#1E90FF", 
    "#4169E1", "#8A2BE2", "#9B30FF", "#FF00FF", "#C71585", 
    "#DC143C", "#FF1493", "#D2691E", "#8B4513", "#A52A2A", 
    "#B22222", "#FF0000", "#8B0000", "#800000", "#006400", 
    "#228B22", "#6B8E23", "#808000", "#DDA0DD", "#EE82EE"
]  
random.shuffle(colors)
colors = [ "grey" ] + colors


In [ ]:
inputFilePath = os.path.join('..', 'results', 'cboehm_10_10000.json')
with open(inputFilePath, 'r') as file:
    dataDct = json.load(file)

In [ ]:
outputDf = pd.DataFrame(dataDct['output'])
nPoints = outputDf.shape[0]
lastNProcessed = 0
for idx, stepDct in enumerate(dataDct['steps']):
    outputDf[f'is_core_{idx:04.0f}'] = dataDct['steps'][idx]['is_core']
    outputDf[f'cluster_id_{idx:04.0f}'] = dataDct['steps'][idx]['clusters']
    curNProcessed = dataDct['steps'][idx]['endCur']
    outputDf[f'processed_{idx:04.0f}'] = [ (2 if i >= lastNProcessed else 1) if i < curNProcessed else 0 for i in range(0,nPoints) ]
    lastNProcessed = curNProcessed
nSteps = len(dataDct['steps'])

In [ ]:
outputDf

In [ ]:
outputDir = os.path.join('..', 'plots')
os.makedirs(outputDir, exist_ok=True)


def createPlot(vs):
    fig, ax = plt.subplots(figsize=(20,15))
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.xlim(0,40)
    plt.ylim(0,25)
    
    xs = outputDf['x']
    ys = outputDf['y']

    clusterColName = f'cluster_id_{vs:04.0f}'
    processedDf = outputDf[outputDf[f'processed_{vs:04.0f}']>=1]
    unprocessedDf = outputDf[outputDf[f'processed_{vs:04.0f}']==0]
    plt.scatter(unprocessedDf['x'], unprocessedDf['y'], s=60, color='black', marker='x')
    for cl, grp in processedDf.groupby(clusterColName):
        xsCoreCurProcessed = grp[(grp[f'is_core_{vs:04.0f}'] == 1) * (grp[f'processed_{vs:04.0f}'] == 2)]['x']
        ysCoreCurProcessed = grp[(grp[f'is_core_{vs:04.0f}'] == 1) * (grp[f'processed_{vs:04.0f}'] == 2)]['y']
        xsCoreProcessed = grp[(grp[f'is_core_{vs:04.0f}'] == 1) * (grp[f'processed_{vs:04.0f}'] == 1)]['x']
        ysCoreProcessed = grp[(grp[f'is_core_{vs:04.0f}'] == 1) * (grp[f'processed_{vs:04.0f}'] == 1)]['y']
        xsNonCore = grp[grp[f'is_core_{vs:04.0f}'] == 0]['x']
        ysNonCore = grp[grp[f'is_core_{vs:04.0f}'] == 0]['y']

        plt.scatter(xsNonCore, ysNonCore, s=60, facecolor='none', marker='o', linewidth=2, color=colors[cl % len(colors)])
        plt.scatter(xsCoreProcessed, ysCoreProcessed, s=60, color=colors[cl % len(colors)])
        plt.scatter(xsCoreCurProcessed, ysCoreCurProcessed, s=60, color=colors[cl % len(colors)], edgecolor='black')
        
    nProcessed = processedDf.shape[0]
    if vs == 0:
        explanation = 'initial'
    elif vs % 2 == 1:
        explanation = f'{nProcessed:.0f} points processed, before collision handling'
    else:
        explanation = f'{nProcessed:.0f} points processed, after collision handling'
    ax.text(1, 24, explanation, fontsize=20)
    
    return fig


    #plt.show()

    
for vs in range(0,nSteps):
    fig = createPlot(vs)

    fig.savefig(os.path.join(outputDir, f'cboehm_{vs:03.0f}.png'), format='png', bbox_inches='tight')

In [ ]:
set(outputDf['cluster_id'])